In [ ]:
import nltk 
nltk.download("stopwords")

In [91]:
from __future__ import print_function

import os
from ftplib import FTP
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
# import nltk
from nltk.corpus import stopwords
from nltk import tokenize
import string
import numpy as np
import re
import glob
from io import open


In [90]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/rhaggerty/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
!ls

Bill Text Train-Test Creation.ipynb get_ftp_files.py
README.md                           tx
document_clustering.ipynb


In [32]:
ftp = FTP('ftp.legis.state.tx.us')
ftp.login()

'230 Anonymous user logged in.'

In [13]:
#Directory structure: /bills/<legislative session>/<document type>/<text format>/<bill type>/<grouping of 100 bills>
ftp.cwd('/bills/851/billtext/html/house_bills/HB00001_HB00099')
files = ftp.nlst()
print files

['HB00001E.htm', 'HB00001H.htm', 'HB00001I.htm', 'HB00002E.htm', 'HB00002H.htm', 'HB00002I.htm', 'HB00003I.htm', 'HB00004H.htm', 'HB00004I.htm', 'HB00005I.htm', 'HB00006I.htm', 'HB00007E.htm', 'HB00007F.htm', 'HB00007H.htm', 'HB00007I.htm', 'HB00007S.htm', 'HB00009E.htm', 'HB00009H.htm', 'HB00009I.htm', 'HB00010E.htm', 'HB00010H.htm', 'HB00010I.htm', 'HB00011E.htm', 'HB00011H.htm', 'HB00011I.htm', 'HB00012I.htm', 'HB00013E.htm', 'HB00013F.htm', 'HB00013H.htm', 'HB00013I.htm', 'HB00013S.htm', 'HB00014H.htm', 'HB00014I.htm', 'HB00015H.htm', 'HB00015I.htm', 'HB00016H.htm', 'HB00016I.htm', 'HB00017H.htm', 'HB00017I.htm', 'HB00018H.htm', 'HB00018I.htm', 'HB00019H.htm', 'HB00019I.htm', 'HB00020E.htm', 'HB00020H.htm', 'HB00020I.htm', 'HB00021E.htm', 'HB00021F.htm', 'HB00021H.htm', 'HB00021I.htm', 'HB00021S.htm', 'HB00022H.htm', 'HB00022I.htm', 'HB00023E.htm', 'HB00023H.htm', 'HB00023I.htm', 'HB00024H.htm', 'HB00024I.htm', 'HB00025E.htm', 'HB00025H.htm', 'HB00025I.htm', 'HB00026E.htm', 'HB0002

## Download html files from FTP

In [51]:
# The naming convention for individual documents is: <bill chamber><bill type><bill suffix><bill version>
# File Name	Description
# <bill chamber>	H=House, S=Senate
# <bill type>	B=Bill, J=Joint Resolution, R=Resolution, C=Concurrent Resolution
# <bill suffix>	5-digit number
# <bill version>	I=Introduced, S=Senate Committee Report, H=House Committee Report, E=Engrossed, F=Enrolled


def is_empty(file):
    file_size=os.stat(file).st_size
    if file_size == 0:
        print("The file {} is empty".format(file))
        return True
    return False

In [88]:
for file in files:
    try:
        local_filename = os.path.join(local_dir, file)
        print("Getting filename " + file)
        ftp.retrbinary('RETR %s' % file, open(local_filename, 'wb').write)
        print("Saving at %s" % local_filename)
    except Exception, err:
        print(err)
        if (is_empty(local_filename)):
            os.remove(local_filename)
        continue

Getting filename HB00001E.htm
[Errno 49] Can't assign requested address
Getting filename HB00001H.htm
[Errno 32] Broken pipe
Getting filename HB00001I.htm
[Errno 32] Broken pipe
Getting filename HB00002E.htm
[Errno 32] Broken pipe
Getting filename HB00002H.htm
[Errno 32] Broken pipe
Getting filename HB00002I.htm
[Errno 32] Broken pipe
Getting filename HB00003I.htm
[Errno 32] Broken pipe
Getting filename HB00004H.htm
[Errno 32] Broken pipe
Getting filename HB00004I.htm
[Errno 32] Broken pipe
Getting filename HB00005I.htm
[Errno 32] Broken pipe
Getting filename HB00006I.htm
[Errno 32] Broken pipe
Getting filename HB00007E.htm
[Errno 32] Broken pipe
Getting filename HB00007F.htm
[Errno 32] Broken pipe
Getting filename HB00007H.htm
[Errno 32] Broken pipe
Getting filename HB00007I.htm
[Errno 32] Broken pipe
Getting filename HB00007S.htm
[Errno 32] Broken pipe
Getting filename HB00009E.htm
[Errno 32] Broken pipe
Getting filename HB00009H.htm
[Errno 32] Broken pipe
Getting filename HB00009I.h

In [4]:
local_dir = '/Users/rhaggerty/remote/bills/tx/data/'

In [180]:
output_dir

'/Users/rhaggerty/remote/bills/tx/data/text'

In [128]:
# #make sure html file isn't empty, if it is, remove it
# def prettify_soup(doc_name):
# #     'HB00098I'
#     bill_no = doc_name.split('/')[-1].rstrip('.htm')
#     output_dir = os.path.join(local_dir, 'text/')
    
#     if (is_empty(doc_name)):
#         os.remove(doc_name)
#     else:
#         with open(doc_name, 'rt') as in_file:
#             soup = BeautifulSoup(in_file, 'html.parser') 
#             pretty_text = soup.get_text(' ', strip=True)
# #             twopretty = pretty_text.prettify()
# #             print(pretty_text)
#             with open(output_dir + bill_no + 'TEST2.txt', 'w+') as out_file:
# #                 out_file.write(pretty_text.encode('utf8'))
#                 out_file.write(pretty_text)
    

#make sure html file isn't empty, if it is, remove it
def prettify_soup(doc_name):
#     'HB00098I'
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    output_dir = os.path.join(local_dir, 'text/')
    
    if (is_empty(doc_name)):
        os.remove(doc_name)
    else:    
    
        with open(doc_name, 'rt') as in_file:
            soup = BeautifulSoup(in_file, 'html.parser') 
            pretty_text = soup.get_text(' ', strip=True)
#             print(pretty_text)
            prettytwo = pretty_text.encode('utf8')
            with open(output_dir + bill_no + 'TEST3.txt', 'w+') as out_file:
                out_file.write(prettytwo)

In [129]:
# for doc_name in glob.glob(local_dir + 'html/*.htm'):
# #     doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00098I.htm'
#     prettify_soup(doc_name)
    

doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00001H.htm'
# output_dir = './tx/data/text/'
prettify_soup(doc_name)

# with open(doc_name, 'rt') as in_file:
#     soup = BeautifulSoup(in_file, 'html.parser')
#     pretty_text = soup.get_text(' ', strip=True)
# #     twopretty = pretty_text.encode('utf8')
# #     pretty_text
#     with open(output_dir + bill_no + '.txt', 'w+', encoding='utf-8') as out_file:
#         out_file.write(pretty_text)

TypeError: must be unicode, not str

In [133]:

#make sure html file isn't empty, if it is, remove it
def prettify_soup(doc_name):
#     'HB00098I'
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    output_dir = os.path.join(local_dir, 'text/')
    
    if (is_empty(doc_name)):
        os.remove(doc_name)
    else:
        with open(doc_name) as in_file:
            soup = BeautifulSoup(in_file, 'html.parser') 
            pretty_text = soup.get_text(' ', strip=True)
            print(pretty_text)
#             with open(output_dir + bill_no + '.txt', 'w+') as out_file:
#                 out_file.write(pretty_text.encode('utf8'))

doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00001H.htm'
# output_dir = './tx/data/text/'
prettify_soup(doc_name)

85(1) HB 1 - House Committee Report version - Bill Text td { font-family: Courier, Arial, sans-serif; font-size: 10pt; } table { empty-cells:show; } 85S10968 MCK-D By: Gonzales of Williamson, Burkett, Raymond, H.B. No. 1 Flynn, Thompson of Harris A BILL TO BE ENTITLED AN ACT relating to avoiding the abolishment of certain agencies subject to the Texas Sunset Act on September 1, 2017. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS: SECTION 1.  Section 151.004, Occupations Code, is amended to read as follows: Sec. 151.004.  APPLICATION OF SUNSET ACT.  The Texas Medical Board is subject to Chapter 325, Government Code (Texas Sunset Act).  Unless continued in existence as provided by that chapter, the board is abolished and this subtitle and Chapters 204, 205, 206, 601, 602, 603, and 604 expire September 1, 2019 [ 2017 ]. SECTION 2.  Section 501.005, Occupations Code, is amended to read as follows: Sec. 501.005.  APPLICATION OF SUNSET ACT.  The Texas State Board of Examiners of Psy

In [134]:
days_file = open('/Users/rhaggerty/remote/bills/tx/data/text/HB00001H.txt','rt')
days_file.readline()

u'85(1) HB 1 - House Committee Report version - Bill Text td { font-family: Courier, Arial, sans-serif; font-size: 10pt; } table { empty-cells:show; } 85S10968 MCK-D By:\xa0Gonzales of Williamson, Burkett, Raymond, H.B.\xa0No.\xa01 Flynn, Thompson of Harris A BILL TO BE ENTITLED AN ACT relating to avoiding the abolishment of certain agencies subject to the Texas Sunset Act on September 1, 2017. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS: SECTION\xa01.\xa0\xa0Section 151.004, Occupations Code, is amended to read as follows: Sec.\xa0151.004.\xa0\xa0APPLICATION OF SUNSET ACT.  The Texas Medical Board is subject to Chapter 325, Government Code (Texas Sunset Act).  Unless continued in existence as provided by that chapter, the board is abolished and this subtitle and Chapters 204, 205, 206, 601, 602, 603, and 604 expire September 1, 2019 [ 2017 ]. SECTION\xa02.\xa0\xa0Section 501.005, Occupations Code, is amended to read as follows: Sec.\xa0501.005.\xa0\xa0APPLICATION OF SUNSET 

In [26]:
!ls /Users/rhaggerty/remote/bills/tx/data/html/HB00007H.htm

ls: /Users/rhaggerty/remote/bills/tx/data/html/HB00007H.htm: No such file or directory


In [18]:
doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00007H.htm'
is_empty(doc_name)

The file /Users/rhaggerty/remote/bills/tx/data/html/HB00007H.htm is empty


True

In [19]:
os.remove(doc_name)

In [21]:
!ls doc_name

ls: doc_name: No such file or directory


In [ ]:
'/Users/rhaggerty/remote/bills/tx/data/

## Gettin' summaries

In [34]:
# if ends in period, then:
# summary = soup.find(text=re.compile('relating'))
# summary
# else get next line and append
# summary.next_element.next_element.next_element.get_text()

doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00002E.htm'
with open(doc_name) as in_file:
    soup = BeautifulSoup(in_file, 'html.parser')
    summary = soup.find(text=re.compile(r'relating.*\.', re.MULTILINE))
    if summary[-1] == '.':
    else:
        next_line = (summary.next_element.next_element.next_element.get_text("", strip=True))
        full_summary = summary + next_line
        print(full_summary)

TypeError: 'NoneType' object has no attribute '__getitem__'

In [92]:
#recursive function for getting title sentence
def get_summary(line):
    if line[-1] == ".":
        return line
    else:
#         return 'nothin'
        line = line.next_element.next_element.next_element.get_text("", strip=True)
        return line + get_summary(line)
        return line
#     return line


In [93]:
doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00002E.htm'
with open(doc_name) as in_file:
    soup = BeautifulSoup(in_file, 'html.parser')
    full_summary = get_summary(summary)
    print(full_summary)

and the Texas State Board of Examiners of Psychologists from the


In [174]:
# for element in summary.next_elements:
#     print(repr(element))
# print(summary)
# print(summary.next_element.next_element.next_element.get_text("", strip=True))
# summary.next_element.next_element.next_element)
# summary.next_element.next_element.next_element.get_text("", strip=True)
summary.next_element.next_element.next_element[1]

KeyError: 1

In [167]:
# soup.find_all(text=re.compile('relating'))
# soup.get_text("", strip=True)
# soup.find(text=re.compile('relating'))
# for child in soup.td.children:
#     print(child.name)

# summary.next_sibling.next_sibling.next_sibling.next_sibling

# soup = BeautifulSoup(html)
tag = soup.findAll('td') #all "td" tag in a list
# print(tag)
for id, tg in enumerate(tag):# i want to go through each piece of TR and print out the values
    print(id, tg)
#     tag_a = tg[id].find('td')
#     for st in tag_a: #to get string 1, string 2, etc
# #         print(st.text[0]) #string 1
#         print(st.text[1]) #string 2
#         print(st.text[2]) #string 3
# #         print st['href'] #/info/12345

0 <td valign="top" width="65"> </td>
1 <td colspan="2" valign="top" width="615">85S11697 BEE-D</td>
2 <td colspan="3"> </td>
3 <td valign="top" width="65"> </td>
4 <td align="left" valign="top" width="450">
		By: Gonzales of Williamson</td>
5 <td align="right" valign="top" width="165">H.B. No. 2</td>
6 <td colspan="3"> </td>
7 <td colspan="3"> </td>
8 <td valign="top" width="65">
<meta contents="" name="PGLN"/> 		
			</td>
9 <td align="center" colspan="2" valign="top" width="615">
<center>A BILL TO BE ENTITLED</center>
</td>
10 <td valign="top" width="65">
<meta contents="1-1" name="PGLN"/> 
			</td>
11 <td align="center" colspan="2" valign="top" width="615">
<center>AN ACT</center>
</td>
12 <td valign="top" width="65">
<meta contents="1-2" name="PGLN"/> 
			</td>
13 <td colspan="2" valign="top" width="615">relating to the repeal of certain riders for the Texas Medical Board </td>
14 <td valign="top" width="65">
<meta contents="1-3" name="PGLN"/> 
			</td>
15 <td colspan="2" valign="to

In [ ]:
# def get_siblings_to_next_anchor(anchor):
# """Anchor Tag -> element list

# Given an anchor element, returns all the nextSiblings elements up to
# (but not including) the next anchor as a list of either Tags or
# NavigatableStrings."""

#     elt = anchor.nextSibling
#     results = []
#     while (elt != None) and (not is_anchor(elt)):
#         results.append(elt)
#     elt = elt.nextSibling
# return results


# def is_anchor(elt):
# """element -> boolean
# Returns true if the element is an anchor Tag."""

#     if isinstance(elt, NavigableString):
#         return False
#     else:
#         return elt.name == 'a'

In [57]:
summary_list = []
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser')
        summary = soup.find(text=re.compile('relating'))
        full_summary = get_summary(summary)
        print(doc_name, full_summary)
#     summary_list.append(full_summary)
# print(summary_list)
#def fib(n):
#     if n > 1:
#         return fib(n-1) + fib(n-2)
#     return n

# for i in range (20):
#     print i, fib(i)

/Users/rhaggerty/remote/bills/tx/data/html/HB00001E.htm relating to avoiding the abolishment of certain agencies subject to the Texas Sunset Act on September 1, 2017.
/Users/rhaggerty/remote/bills/tx/data/html/HB00001H.htm relating to avoiding the abolishment of certain agencies subject to the Texas Sunset Act on September 1, 2017.
/Users/rhaggerty/remote/bills/tx/data/html/HB00001I.htm relating to avoiding the abolishment of certain agencies subject to the Texas Sunset Act on September 1, 2017.


AttributeError: 'unicode' object has no attribute 'next_element'

In [36]:
summary_list = []
summary_dict = {}
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser')
        title_line = soup.find(text=re.compile(r'relating.*\.', re.MULTILINE))
#         if title_line is not None:
#             if title_line[-1] == '.':
#                 summary = title_line
# #             print(title_line)
#             else:
#                 next_line = (title_line.next_element.next_element.next_element.get_text("", strip=True))
#                     if next_line[-1] == '.':
#                         summary = title_line + next_line
#                     else: 
    summary_list.append(title_line)
print(summary_list)

[None, None, None, None, None, None, u'relating to ad valorem taxation; authorizing fees.', None, None, None, u'relating to municipal annexation.', None, None, None, None, u'collecting accurate information relating to maternal mortality.', u'collecting accurate information relating to maternal mortality.', None, u'relating to maternal morbidity and pregnancy-related deaths.', u'relating to maternal morbidity and pregnancy-related deaths.', u'relating to maternal morbidity and pregnancy-related deaths.', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, u'relating to the public school finance system.', None, u'relating to the public school finance system.', u'relating to the public school finance system.', u'relating to the public school finance system.', None, None, None, None, None, None, None, None, None, None, u'relating to the regulation of groundwater.', u'relating to the regulation of gr

In [32]:
summary_list = []
bill_no_list = []
summary_dict = {}
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser')
        summary = soup.find(text=re.compile(r'relating.*\.', re.MULTILINE))
    summary_list.append(summary)
    bill_no_list.append(bill_no)
bill_summaries = zip(bill_no_list, summary_list)
for bill, summary in bill_summaries:
    summary_dict[bill] = summary
print(summary_dict)

{'HB00197I': None, 'HB00284I': None, 'HB00233I': None, 'HB00105I': u'relating to the minimum wage.', 'HB00150I': None, 'HB00058I': None, 'HB00111H': None, 'HB00005I': None, 'HB00042I': None, 'HB00372I': None, 'HB00331H': None, 'HB00198H': None, 'HB00198I': u'relating to compliance with this section.', 'HB00263I': None, 'HB00159I': None, 'HB00350I': None, 'HB00220I': None, 'HB00315I': None, 'HB00201I': None, 'HB00266I': None, 'HB00177I': None, 'HB00180I': None, 'HB00293I': None, 'HB00283I': None, 'HB00111I': None, 'HB00238I': None, 'HB00157I': None, 'HB00157H': None, 'HB00349I': None, 'HB00118I': None, 'HB00355I': u'relating to the public school finance system.', 'HB00045I': u'information relating to or described by Subsection (f).\xa0\xa0The ', 'HB00312I': None, 'HB00365I': u'relating to regulation of certain credit services organizations.', 'HB00187H': None, 'HB00187I': None, 'HB00270I': None, 'HB00064I': None, 'HB00323I': None, 'HB00012I': None, 'HB00333I': None, 'HB00383I': None, 'H

In [241]:
clean_summary = []
for summ in summary_list:
    if summ is not None:
        clean = re.sub(r'relating to', ' ', summ)
        words = clean.split()
        non_stop = [w for w in words if not w in stopwords.words("english")]
#         print(non_stop)
    clean_summary.append(non_stop)
#     print(clean)
word_set = set([i[0] for i in clean_summary])
word_set

{u'abolition',
 u'actions',
 u'ad',
 u'adjustment',
 u'administration',
 u'adoption',
 u'applicability',
 u'appropriation',
 u'authority',
 u'authorizing',
 u'automatic',
 u'avoiding',
 u'calculation',
 u'compensation',
 u'conflicts',
 u'constitutional',
 u'cost-of-living',
 u'dedicating',
 u'development',
 u'disclosure',
 u'early',
 u'eligibility',
 u'establishment',
 u'ethics',
 u'exemption',
 u'extension',
 u'extraterritorial',
 u'funding',
 u'general',
 u'grant',
 u'limitation',
 u'lobbying',
 u'making',
 u'maternal',
 u'membership',
 u'municipal',
 u'number',
 u'opportunity',
 u'optional',
 u'oversight',
 u'pregnancy',
 u'pregnancy-related',
 u'prevention',
 u'prohibited',
 u'prohibiting',
 u'property',
 u'public',
 u'receipt',
 u'regulation',
 u'reimbursement',
 u'release',
 u'repeal',
 u'reporting',
 u'requiring',
 u'restrictions',
 u'salary',
 u'school',
 u'selection',
 u'signature',
 u'supplemental',
 u'transfer',
 u'transferring',
 u'transportation',
 u'tree',
 u'use'}

In [242]:
word_list = [i[0] for i in clean_summary]
word_list

[u'avoiding',
 u'avoiding',
 u'avoiding',
 u'repeal',
 u'repeal',
 u'funding',
 u'ad',
 u'calculation',
 u'calculation',
 u'limitation',
 u'municipal',
 u'tree',
 u'municipal',
 u'municipal',
 u'tree',
 u'municipal',
 u'maternal',
 u'maternal',
 u'maternal',
 u'maternal',
 u'maternal',
 u'maternal',
 u'pregnancy-related',
 u'pregnancy-related',
 u'pregnancy-related',
 u'general',
 u'reporting',
 u'reporting',
 u'reporting',
 u'reporting',
 u'reporting',
 u'prohibiting',
 u'prohibiting',
 u'lobbying',
 u'lobbying',
 u'ethics',
 u'ethics',
 u'disclosure',
 u'disclosure',
 u'conflicts',
 u'conflicts',
 u'making',
 u'making',
 u'appropriation',
 u'appropriation',
 u'appropriation',
 u'public',
 u'public',
 u'public',
 u'public',
 u'public',
 u'extension',
 u'extension',
 u'grant',
 u'grant',
 u'grant',
 u'salary',
 u'salary',
 u'reimbursement',
 u'reimbursement',
 u'reimbursement',
 u'regulation',
 u'regulation',
 u'regulation',
 u'development',
 u'development',
 u'development',
 u'members

In [245]:
from collections import Counter
favs = Counter(word for word in word_list)
favs.most_common()

[(u'maternal', 7),
 (u'regulation', 6),
 (u'public', 5),
 (u'municipal', 5),
 (u'reporting', 5),
 (u'appropriation', 5),
 (u'limitation', 5),
 (u'making', 5),
 (u'administration', 4),
 (u'school', 4),
 (u'pregnancy-related', 4),
 (u'exemption', 4),
 (u'selection', 3),
 (u'applicability', 3),
 (u'cost-of-living', 3),
 (u'grant', 3),
 (u'development', 3),
 (u'membership', 3),
 (u'repeal', 3),
 (u'salary', 3),
 (u'reimbursement', 3),
 (u'avoiding', 3),
 (u'prevention', 3),
 (u'disclosure', 2),
 (u'prohibiting', 2),
 (u'extraterritorial', 2),
 (u'calculation', 2),
 (u'authority', 2),
 (u'general', 2),
 (u'use', 2),
 (u'transfer', 2),
 (u'establishment', 2),
 (u'lobbying', 2),
 (u'restrictions', 2),
 (u'eligibility', 2),
 (u'conflicts', 2),
 (u'ethics', 2),
 (u'extension', 2),
 (u'tree', 2),
 (u'compensation', 2),
 (u'abolition', 2),
 (u'signature', 2),
 (u'ad', 1),
 (u'constitutional', 1),
 (u'receipt', 1),
 (u'number', 1),
 (u'dedicating', 1),
 (u'pregnancy', 1),
 (u'transportation', 1),


In [223]:

words = [w for w in words if not w in stopwords.words("english")]
print words

[u'  avoiding the abolishment of certain agencies subject to ',
 u'  avoiding the abolishment of certain agencies subject to ',
 u'  avoiding the abolishment of certain agencies subject to ',
 u'  the repeal of certain riders for the Texas Medical Board ',
 u'  the repeal of certain riders for the Texas Medical Board ',
 u'  the funding of certain governmental entities subject to ',
 u'  ad valorem taxation; authorizing fees.',
 u'  the calculation of the ad valorem rollback tax rate of a ',
 u'  the calculation of the ad valorem rollback tax rate of a ',
 u'  a limitation on increases in the appraised value for ad ',
 u'  municipal annexation.',
 u'  a tree planting credit to offset tree mitigation fees ',
 u'  municipal regulation of the removal of trees on private ',
 u'  municipal regulation of the removal of trees on private ',
 u'  a tree planting credit to offset tree mitigation fees ',
 u'  municipal regulation of the removal of trees or ',
 u'  maternal health and safety, preg

In [112]:
pretty_text = soup.get_text(" ", strip=True)
pretty_text

u'85(1) HB 98 - Introduced version - Bill Text td { font-family: Courier, Arial, sans-serif; font-size: 10pt; } table { empty-cells:show; } 85S10371 MK-D By:\xa0Bernal H.B.\xa0No.\xa098 A BILL TO BE ENTITLED AN ACT relating to a school district assigning a mentor teacher to a new classroom teacher. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS: SECTION\xa01.\xa0\xa0Section 21.458, Education Code, is amended by adding Subsections (a-1), (b-1), (f), and (g) and amending Subsections (b) and (d) to read as follows: (a-1) To be assigned as a mentor, a teacher must agree to serve as a mentor teacher for at least two school years. The assignment must begin on the first day of employment of the classroom teacher to whom the mentor teacher is assigned. (b)\xa0\xa0The commissioner shall adopt rules necessary to administer this section, including rules concerning the duties and qualifications of a teacher who serves as a mentor and the number of classroom teachers that may be assigned to

In [16]:
for k,v in summary_dict.items(): 
    if summary_dict[k] is None:
        print(k) 


HB00007H


# Try Again I guess

In [ ]:
summary_list = []
bill_no_list = []
summary_dict = {}
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser')
        summary = soup.find(text=re.compile('relating'))
    summary_list.append(summary)
    bill_no_list.append(bill_no)
bill_summaries = zip(bill_no_list, summary_list)
for bill, summary in bill_summaries:
    summary_dict[bill] = summary
print(summary_dict)

In [77]:
def solution(S):
    sentences_list = re.split(r'[.!?]', S)
    wc_list = []
    for sentence in sentences_list:
        x = sentence.strip()
        words = re.split('\s', x)
        wc_list.append(words)
    return wc_list
# solution(S)

In [ ]:
# with open('/Users/rhaggerty/remote/bills/tx/data/text/HB00077I.txt') as in_file:
#     text = re.compile('^relating')
#     print(text)
# pattern = re.compile('relating')
# for i, line in enumerate(open('/Users/rhaggerty/remote/bills/tx/data/text/HB00077I.txt')):
# #     for match in re.finditer(pattern, line):
# #         print(i, line)
#     print(line)
        
# for line in open('/Users/rhaggerty/remote/bills/tx/data/text/HB00077I.txt'):
# #     for match in re.finditer(pattern, line):
# #         print(i, line)
#     sentences_list = re.split(r'[.!?]', line)
#     for sentence in sentences_list:
#         print(sentence + '\n')     
        
        
# doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00098I.htm'
# output_dir = './tx/data/text/'
# # prettify_soup(doc_name)

# with open(doc_name, 'rt') as in_file:
#     soup = BeautifulSoup(in_file, 'html.parser')
#     pretty_text = soup.get_text(' ', strip=True)
# #     twopretty = pretty_text.encode('utf8')
# #     pretty_text
#     with open(output_dir + bill_no + '.txt', 'w+', encoding='utf-8') as out_file:
#         out_file.write(pretty_text)

In [102]:
# with open('/Users/rhaggerty/remote/bills/tx/data/text/HB00394I.txt') as in_file:
#         print(str(in_file))
        
        
days_file = open('/Users/rhaggerty/remote/bills/tx/data/text/HB00394I.txt','rt')
thing = days_file.read()
print(thing)

85(1) HB 98 - Introduced version - Bill Text td { font-family: Courier, Arial, sans-serif; font-size: 10pt; } table { empty-cells:show; } 85S10371 MK-D By: Bernal H.B. No. 98 A BILL TO BE ENTITLED AN ACT relating to a school district assigning a mentor teacher to a new classroom teacher. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS: SECTION 1.  Section 21.458, Education Code, is amended by adding Subsections (a-1), (b-1), (f), and (g) and amending Subsections (b) and (d) to read as follows: (a-1) To be assigned as a mentor, a teacher must agree to serve as a mentor teacher for at least two school years. The assignment must begin on the first day of employment of the classroom teacher to whom the mentor teacher is assigned. (b)  The commissioner shall adopt rules necessary to administer this section, including rules concerning the duties and qualifications of a teacher who serves as a mentor and the number of classroom teachers that may be assigned to a mentor .  The rules con

In [104]:
sentences_list = tokenize.sent_tokenize(thing)
sentences_list[2]

In [107]:
sentences_list[2]

u'98 A BILL TO BE ENTITLED AN ACT relating to a school district assigning a mentor teacher to a new classroom teacher.'

In [16]:
local_dir = '/Users/rhaggerty/remote/bills/tx/data/'
for doc_name in glob.glob(local_dir + 'text/*.txt'):
    bill_no = doc_name.split('/')[-1].rstrip('.txt')
    with open(doc_name) as in_file:
        text=re.compile('^relating.*\.')
        print(text)

HB00001E
HB00001H
HB00001I
HB00002E
HB00002H
HB00002I
HB00003I
HB00004H
HB00004I
HB00005I
HB00006I
HB00007E
HB00007F
HB00007H
HB00007I
HB00007S
HB00009E
HB00009H
HB00009I
HB00010E
HB00010H
HB00010I
HB00011E
HB00011H
HB00011I
HB00012I
HB00013E
HB00013F
HB00013H
HB00013I
HB00013S
HB00014H
HB00014I
HB00015H
HB00015I
HB00016H
HB00016I
HB00017H
HB00017I
HB00018H
HB00018I
HB00019H
HB00019I
HB00020E
HB00020H
HB00020I
HB00021E
HB00021F
HB00021H
HB00021I
HB00021S
HB00022H
HB00022I
HB00023E
HB00023H
HB00023I
HB00024H
HB00024I
HB00025E
HB00025H
HB00025I
HB00026E
HB00026H
HB00026I
HB00027E
HB00027H
HB00027I
HB00028E
HB00028H
HB00028I
HB00029I
HB00030E
HB00030F
HB00030H
HB00030I
HB00030S
HB00031I
HB00032E
HB00032H
HB00032I
HB00032S
HB00033H
HB00033I
HB00041I
HB00042I
HB00043I
HB00044I
HB00045I
HB00046I
HB00047H
HB00047I
HB00048I
HB00049I
HB00050I
HB00051I
HB00052I
HB00053I
HB00054I
HB00055I
HB00056I
HB00057I
HB00058I
HB00059I
HB00060H
HB00060I
HB00061I
HB00062I
HB00063I
HB00064I
HB00065I
HB00066I
H

In [37]:
train = load_files("data/")

In [38]:
train.filenames

array([], dtype=float64)

In [39]:
train

{'DESCR': None,
 'data': [],
 'filenames': array([], dtype=float64),
 'target': array([], dtype=float64),
 'target_names': []}

In [ ]:
print("\n".join(train.data[0].split("\n")[:3]))

In [ ]:
print(train.target_names[train.target[0]])

In [10]:
import pdb

In [11]:
def debug(text, context=""):
#     """Debug."""
    if DEBUG_ENABLED:
        print '[toggle_single_line_css]:' + text
    if PRINT_CONTEXT and context != "":
        print '>>> ' + context
#     pass

In [12]:
debug('blah', context="")

NameError: global name 'DEBUG_ENABLED' is not defined